#**CORRELACIÓN PUNTOS MEDIDA EQUIVALENTES A UNA RUTA GENERADA POR GOOGLE MAPS**

El presente modelo se realiza para correlaccionar los puntos de la ruta generada por GoogleMaps con los puntos de medida de trafico de la ciudad de Madrid utilizados en el modelo de predicción de la carga de tráfico. 

#0.Inicialización

In [ ]:
import numpy as np
import pandas as pd

#1.Datos

Se importa el fichero que contiene los puntos de ruta de google maps

In [ ]:
ruta=pd.read_csv('/content/drive/MyDrive/Proyecto PSIA/Notebooks/Ruta.csv', sep=';', header=0, index_col=False)
ruta.count()

latitud     248
longitud    248
id          248
dtype: int64

Se importa el fichero con la ubicación de los puntos de medida más reciente

In [ ]:
puntos=pd.read_csv('/content/drive/MyDrive/Proyecto PSIA/Notebooks/pmed_ubicacion_12-2019_decimal.csv', sep=';', header=0, index_col=False)
puntos.count()

id          4253
longitud    4253
latitud     4253
dtype: int64

In [ ]:
import geopy 
from geopy.geocoders import ArcGIS
from geopy.distance import geodesic

Se actualiza la versión de pandas para poder utilizar merge cross

In [ ]:
pd.__version__
!pip install pandas==1.2.3

#2.Manipulación Datos

se une cada registro de los puntos de medida con cada registro de los puntos de ruta

In [ ]:
distancia=puntos.merge(ruta,how='cross')
distancia.head()

,id_x,longitud_x,latitud_x,latitud_y,longitud_y,id_y
0,3840,-3.688323,40.430502,40.46570,-3.68971,1
1,3840,-3.688323,40.430502,40.46567,-3.68966,2
2,3840,-3.688323,40.430502,40.46560,-3.68971,3
3,3840,-3.688323,40.430502,40.46554,-3.68975,4
4,3840,-3.688323,40.430502,40.46551,-3.68976,5


#3.Cálculo de la distancia Puntos de Medida con Puntos Ruta

Se crea una función para calcular la distancia entre dos coordenadas (longitud, latitud) utilizando la librería geopy.
Se aplica esta función a cada registro de la tabla que contiene todas las combinaciones de punto de medida con punto de ruta.

In [ ]:
def calculo(distancia):
  coorpuntos=(distancia['latitud_x'],distancia['longitud_x'])
  coorruta=(distancia['latitud_y'],distancia['longitud_y'])
  dist=abs((geodesic(coorruta,coorpuntos).km))
  return dist

distancia['distancia']=distancia.apply(lambda distancia: calculo(distancia), axis=1)
distancia.head()


,id_x,longitud_x,latitud_x,latitud_y,longitud_y,id_y,distancia
0,3840,-3.688323,40.430502,40.46570,-3.68971,1,3.910285
1,3840,-3.688323,40.430502,40.46567,-3.68966,2,3.906830
2,3840,-3.688323,40.430502,40.46560,-3.68971,3,3.899186
3,3840,-3.688323,40.430502,40.46554,-3.68975,4,3.892630
4,3840,-3.688323,40.430502,40.46551,-3.68976,5,3.889327


Se agrupa la tabla por identificador de punto de ruta y para cada uno de estos puntos nos quedamos con el valor mínimo de la distancia

In [ ]:
distancia_min=distancia.groupby(['id_y']).min(numeric_only=True)
distancia_min


,id_x,longitud_x,latitud_x,latitud_y,longitud_y,distancia
id_y,,,,,,
1,0,-3.836943,40.332454,40.46570,-3.68971,0.074289
2,0,-3.836943,40.332454,40.46567,-3.68966,0.078565
3,0,-3.836943,40.332454,40.46560,-3.68971,0.074994
4,0,-3.836943,40.332454,40.46554,-3.68975,0.072891
5,0,-3.836943,40.332454,40.46551,-3.68976,0.072922
...,...,...,...,...,...,...
244,0,-3.836943,40.332454,40.42338,-3.71086,0.057135
245,0,-3.836943,40.332454,40.42322,-3.71116,0.026926
246,0,-3.836943,40.332454,40.42314,-3.71134,0.010830


Como al hacer la operación anterior perdemos la verdadera referencia del identificador del punto de medida es necesario unir la distancia calculada con la tabla original para recuperar esta información.

Para ello primeramente eliminamos todos los campos que no nos interesan de la tabla de distancias

In [ ]:
distancia_min_ok=distancia_min.drop(['id_x','longitud_x','longitud_y','latitud_x','latitud_y'], axis=1)
distancia_min_ok

,distancia
id_y,
1,0.074289
2,0.078565
3,0.074994
4,0.072891
5,0.072922
...,...
244,0.057135
245,0.026926
246,0.010830


#4.Obtención resultados Puntos equivalente ruta

Hacemos la unión para localizar la identificación del punto

In [ ]:

distancia_ok=pd.merge(distancia_min_ok,distancia, on=['distancia'],how='left')

In [ ]:
distancia_ok['repeticiones']=0
distancia_ok_sum=distancia_ok.groupby('id_x', as_index=False).agg({'distancia':'min','repeticiones':'count','longitud_x':'min','latitud_x':'min'})
distancia_ok_sum.to_csv('/content/drive/MyDrive/Proyecto PSIA/Ptos_segun_ruta.csv')
distancia_ok_sum.head()

,id_x,distancia,repeticiones,longitud_x,latitud_x
0,3397,0.095523,3,-3.713966,40.439020
1,3421,0.072891,9,-3.690586,40.465693
2,3423,0.073100,1,-3.690087,40.462787
3,3511,0.011521,5,-3.710894,40.439773
4,3619,0.014490,4,-3.713161,40.435190


Eliminamos los puntos cuya distancia sea superior a 70 metros

In [ ]:
ptosruta = distancia_ok_sum.drop(distancia_ok_sum[distancia_ok_sum['distancia']>0.07].index)
ptosruta.to_csv('/content/drive/MyDrive/Proyecto PSIA/Notebooks/Ptos_segun_ruta_ajustada.csv')